In [ ]:
#################################################
################  Import things #################
#################################################

import numpy as np
import timeit
import random
import copy
import matplotlib.pyplot as plt
from sklearn import datasets
import pickle
import glycowork
import CandyCrunch

# need to previously `run pip3 install esm`
import esm.pretrained
from glycowork.ml.inference import get_esm1b_representations, get_lectin_preds
from glycowork.ml.models import prep_model
from glycowork.glycan_data.loader import glycan_binding as gb


model, alphabet = esm.pretrained.esm1b_t33_650M_UR50S() # will need to download if the first time


#@markdown # Step 1: Install all neccessary packages

from IPython.utils.io import capture_output
import os
import subprocess

# from google.colab import drive
# drive.mount('/content/drive')

try:
  with capture_output() as captured:
    !apt install libcairo2-dev
    !pip install CandyCrunch[draw]
    from glycowork.motif.draw import GlycoDraw
    from CandyCrunch.prediction import *
    from CandyCrunch.analysis import *
    !apt-get install wkhtmltopdf
    !pip install imgkit
    !pip install IPython~=7.34.0
    from IPython.core.display import display,HTML
    !pip install imgkit==1.2.3
    import imgkit
    !pip install PIL~=8.4.0
    from PIL import Image
    !pip install bokeh==2.4.3
    import bokeh
    import io
    import warnings
    import base64

except subprocess.CalledProcessError as captured:
  print(captured)
  raise

def prob_dist(vals):
    return vals / np.sum(vals)


In [14]:
#@markdown # Step 2: Select your input file and predict structures
#@markdown **Note:** Do not change any fields in Step 2 if you wish to use the example spectra
#@markdown ### Enter the directory address to your input files
directory_address  = '/Users/csfloyd/Dropbox/Projects/GlycanAnalysis/Data/'  #@param {type:"string"}
#@markdown ### Enter the name of your input file (as one of the formats .mzML, .mzXML, or .xlsx)
filename = 'data/GPST000350/JC_200217P1N_200218002345.xlsx'  #@param {type:"string"}
example = directory_address + filename
if example == 'drive/MyDrive/example_folder/example_file.mzML':
  try:
    with capture_output() as captured:
      !wget https://raw.githubusercontent.com/BojarLab/CandyCrunch/main/examples/JC_171002Y1_extracted_spectra.xlsx
      spectra_filepath = '/content/JC_171002Y1_extracted_spectra.xlsx'
  except subprocess.CalledProcessError as captured:
    print(captured)
    raise
else:
  spectra_filepath = ''
  if ".xlsx" in filename:
    spectra_filepath = directory_address + filename
  else:
    if  "." in filename:
      filename = filename.split('.')[0]
    input_address = directory_address + filename
    for extension,extraction_function in [('.mzML',process_mzML_stack),('.mzXML',process_mzXML_stack)]:
      if os.path.isfile(input_address + extension):
        if not os.path.isfile(filename + extension + '.xlsx'):
          extraction_function(input_address + extension,intensity=True).to_excel(filename+extension+'.xlsx',index=False)
        spectra_filepath = filename + extension + '.xlsx'
        break

#@markdown ### Select the model parameters:
glycan_class = 'O' #@param ["O", "N", "lipid", "free"]
mode = 'negative' #@param ["negative", "positive"]
liquid_chromatography = 'PGC' #@param ["PGC", "C18", "other"]
trap = 'linear' #@param ["linear", "orbitrap", "amazon", "ToF", "QToF", "other"]
modification = 'reduced' #@param ["reduced", "permethylated", "2AA", "2AB" , "custom"]
#@markdown ##### custom_modification_mass is only passed if modification is set to 'custom'
custom_modification_mass = 0 #@param {type: "number"}

if modification == 'custom':
  mass_tag = custom_modification_mass
else:
  mass_tag = None

warnings.filterwarnings("ignore", category=RuntimeWarning)
df_out,spectra_out = wrap_inference(spectra_filepath, glycan_class,
                                    mode = mode, modification = modification, mass_tag = mass_tag, lc = liquid_chromatography, trap = trap,
                                    spectra=True,experimental=False,supplement=False)

glycan_pred_list = [x[0][0] if x else [] for x in df_out['predictions'].tolist()]
glycan_probs_list = [f'{round(x[0][1]*100,2)}%' if x else 'N/a' for x in df_out['predictions'].tolist()]
glycan_img_list = [GlycoDraw(x).as_svg() if x else '' for x in glycan_pred_list]

glycan_all_preds = []
for preds in df_out['predictions'].tolist():
  glycan_all_preds.append([x[0] for x in preds])
["<br>".join(x) for x in glycan_all_preds]
alt_preds = [x[1:] for x in glycan_all_preds]

glycan_all_probs = []
for preds in df_out['predictions'].tolist():
  glycan_all_probs.append([round(x[1]*100,2) for x in preds])
glycan_all_probs_string = [[str(x)+'%' for x in y] for y in glycan_all_probs]
["<br>".join(x) for x in glycan_all_probs_string]

display_df = df_out.reset_index()
display_df = display_df[[x for x in display_df.columns if x not in ['top_fragments','adduct']]]
display_df['predictions'] = glycan_pred_list
display_df['predicted_snfg'] = glycan_img_list
display_df['prediction_probability'] = glycan_probs_list
display_df = display_df.rename(columns={"index": "m/z","predictions": "predicted_IUPAC",})
if 'rel_abundance' in df_out:
  display_df = display_df.rename(columns={"rel_abundance": "abundance"})
  display_df = display_df[['m/z', 'composition', 'predicted_IUPAC', 'ppm_error', 'prediction_probability', 'num_spectra', 'charge', 'RT', 'abundance', 'predicted_snfg']]
  display_df['abundance'] = [round(x,2) for x in display_df['abundance'].tolist()]
else:
  display_df = display_df[['m/z', 'composition', 'predicted_IUPAC', 'ppm_error', 'prediction_probability', 'num_spectra', 'charge', 'RT','predicted_snfg']]
display_df.index.name = 'prediction ID'
display_df['m/z'] = [round(x,2) for x in display_df['m/z'].tolist()]
display_df['predicted_IUPAC'] = ["<br>".join(x) for x in glycan_all_preds]
display_df['prediction_probability'] = ["<br>".join(x) for x in glycan_all_probs_string]
display_df['alternative_snfg'] = [[GlycoDraw(x).as_svg() for x in shot if GlycoDraw(x)] for shot in alt_preds]

def svg_to_base64(svg):
    svg_bytes = io.BytesIO(svg.encode())
    svg_data = svg_bytes.read()
    return 'data:image/svg+xml;base64,' + base64.b64encode(svg_data).decode('utf-8')

def format_name(name):
    return f'<span style="font-size: 16px">{name}</span>'

def format_main_image(main_image):
    return f'<div style="text-align: center"><img src="{svg_to_base64(main_image)}" /></div>'

def format_alt_images(alt_images):
    svg_list = []
    for alt_image in alt_images:
      svg_list.append(f'<div style="text-align: center; margin-bottom: 10px;"><img src="{svg_to_base64(alt_image)}" /></div>')
    return f''.join(svg_list)

def svg_list_to_html(svg_list):
    cell_html = ''
    for svg_file in svg_list:
        svg_base64 = svg_to_base64(svg_file)
        cell_html += f'<img src="data:image/svg+xml;base64,{svg_base64}" style="display:block; margin: 10px 0;">'
    return cell_html

format_dict = {'predictions':format_name, 'predicted_snfg': format_main_image, 'alternative_snfg': format_alt_images}

html_table = display_df.to_html(escape=False, formatters=format_dict)
html_table = html_table.replace('<th>', '<th style="font-size: 20px">')

new_style = """
<style>
table {
  border-collapse: collapse;
}

th {
  text-align: center;
  background-color: #f2f2f2;
  padding: 8px;
}

td {
  text-align: center;
  padding: 8px;
}

.image {
  text-align: center;
}

.image img {
  width: 200px;
  height: auto;
}

tr.start-page {
    page-break-before: always;
}
tr.end-page {
    page-break-after: always;
}
</style>
"""

html_table = new_style + html_table


# Display HTML table
# display(HTML(html_table))

Your chosen settings are: O glycans, negative ion mode, reduced glycans, PGC LC, and linear ion trap. If any of that seems off to you, please restart with correct parameters.


In [12]:
display_df['predicted_IUPAC']
display_df['abundance']

prediction ID
0      2.49
1      9.27
2      5.87
3      2.65
4      5.99
5      0.22
6      3.86
7      1.69
8      2.85
9      0.29
10     1.29
11     3.60
12     1.68
13     1.59
14     3.58
15     6.89
16     0.42
17     5.65
18     1.60
19     4.93
20     1.36
21     4.02
22     3.00
23     0.27
24     0.48
25     2.89
26    10.44
27     3.50
28     3.50
29     0.29
30     2.31
31     1.53
Name: abundance, dtype: float64

In [16]:
leor = prep_model('LectinOracle', 1, trained = True)

In [29]:
prot_seq_list = []
prot_seq_list.append(gb[gb['protein'] == "ConA"]['target'].iloc[0])
prot_seq_list.append(gb[gb['protein'] == "WGA"]['target'].iloc[0])

glycan_list = display_df['predicted_IUPAC']
probabilities = prob_dist(display_df['abundance'])

n_gly = len(glycan_list)
n_prots = len(prot_seq_list)
z_score_mat = np.zeros((n_prots, n_gly))

for (p, prot_seq) in enumerate(prot_seq_list):
    rep = get_esm1b_representations([prot_seq], model, alphabet)
    for (g, gly) in enumerate(glycan_list):
        try:
            predictions = get_lectin_preds(prot_seq, [gly], leor, rep)
            z_scores = list([predictions['pred'][i] for i in range(len(predictions['pred']))])
        except:
            print(gly)
            z_scores = [0]
    z_score_mat[p,g] = z_scores[0]

Gal(b1-?)GalNAc(a1-3)GalNAc<br>Gal(b1-3)[GlcNAc(b1-6)]GalNAc
Gal(b1-3)[GlcNAc(b1-6)]GalNAc<br>GlcNAc(a1-4)Gal(b1-3)GalNAc
GalNAc(a1-3)[Neu5Ac(a2-6)]GalNAc<br>GlcNAc(b1-3)[Neu5Ac(a2-6)]GalNAc
GlcNAc(b1-3)[Neu5Ac(a2-6)]GalNAc<br>GalNAc(a1-3)[Neu5Ac(a2-6)]GalNAc
Fuc(a1-2)Gal(b1-3)[GlcNAc(b1-6)]GalNAc<br>Fuc(a1-2)Gal(b1-3)GlcNAc(b1-3)GalNAc<br>Fuc(a1-2)[GalNAc(a1-3)]Gal(b1-3)GalNAc
Man(a1-6)Man(b1-4)GlcNAc(b1-4)GlcNAc<br>Gal(a1-3)Gal(b1-4)GlcNAc(b1-3)GalNAc
HexNAc(?1-?)Gal(b1-3)[GlcNAc(b1-6)]GalNAc<br>Gal(b1-4)GlcNAc(b1-3)[GlcNAc(b1-6)]GalNAc
Fuc(a1-2)Gal(b1-4)GlcNAc(b1-6)[Gal(b1-3)]GalNAc<br>Fuc(a1-2)Gal(b1-3)[Gal(b1-4)GlcNAc(b1-6)]GalNAc<br>Man(a1-3)Man(b1-4)GlcNAc(b1-4)[Fuc(a1-6)]GlcNAc
Fuc(a1-2)Gal(b1-4)GlcNAc(b1-6)[Gal(b1-3)]GalNAc<br>Fuc(a1-3)[Gal(b1-4)]GlcNAc(b1-6)[Gal(b1-3)]GalNAc
Neu5Gc(a2-3)Gal(b1-4)GlcNAc(b1-6)[Gal(b1-3)]GalNAc<br>Neu5Gc(a2-3)Gal(b1-4)GlcNAc(b1-3)Gal(b1-3)GalNAc
HexNAc(?1-?)Gal(b1-4)GlcNAc(b1-6)[GlcNAc(?1-?)Gal(b1-3)]GalNAc<br>GlcNAc(a1-4)Gal(b1-4)GlcNAc(b1-6)[H

In [28]:
z_score_mat

array([[0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.96491241],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.2259388 ]])

In [25]:
glycan_list

prediction ID
0                                       Gal(b1-3)GalNAc
1                                    GlcNAc(b1-3)GalNAc
2                              Fuc(a1-2)Gal(b1-3)GalNAc
3                         GlcNAc(b1-4)[Fuc(a1-6)]GlcNAc
4                         GlcNAc(b1-4)[Fuc(a1-6)]GlcNAc
5     Gal(b1-?)GalNAc(a1-3)GalNAc<br>Gal(b1-3)[GlcNA...
6     Gal(b1-3)[GlcNAc(b1-6)]GalNAc<br>GlcNAc(a1-4)G...
7                       Gal(b1-3)[GlcNAc6S(b1-6)]GalNAc
8                           Neu5Ac(a2-3)Gal(b1-3)GalNAc
9     Neu5Ac(a2-3)HexNAc(?1-?)Gal(?1-?)GalNAc(a1-3)[...
10                   GlcNAc(b1-3)[GlcNAc6S(b1-6)]GalNAc
11    GalNAc(a1-3)[Neu5Ac(a2-6)]GalNAc<br>GlcNAc(b1-...
12    GlcNAc(b1-3)[Neu5Ac(a2-6)]GalNAc<br>GalNAc(a1-...
13    Fuc(a1-2)Gal(b1-3)[GlcNAc(b1-6)]GalNAc<br>Fuc(...
14               Gal(b1-4)GlcNAc(b1-6)[Gal(b1-3)]GalNAc
15    Man(a1-6)Man(b1-4)GlcNAc(b1-4)GlcNAc<br>Gal(a1...
16    HexNAc(?1-?)Gal(b1-3)[GlcNAc(b1-6)]GalNAc<br>G...
17             Fuc(a1-2)Gal(b1-3)[